In [1]:
import pandas as pd
import numpy as np
abs_path = "D:\stocknet-dataset-master\price\preprocessed"
df = pd.read_csv(abs_path + '\AAPL.txt', delimiter = "\t",
                names=["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"])
# df.columns = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]

In [2]:
df = df.iloc[::-1]
df = df.reset_index(drop=True)

In [3]:
df = df[["Date","High","Low","Close"]]
df

,Date,High,Low,Close
0,2012-09-05,0.107768,0.109047,0.097979
1,2012-09-06,0.111639,0.120094,0.107725
2,2012-09-07,0.109697,0.116947,0.105966
3,2012-09-10,0.106800,0.111420,0.076953
4,2012-09-11,0.110742,0.119075,0.096363
...,...,...,...,...
1252,2017-08-28,0.001752,0.013387,0.000438
1253,2017-08-29,-0.008485,0.010219,-0.009104
1254,2017-08-30,0.005463,0.006016,-0.001842
1255,2017-08-31,0.001775,0.007163,0.000796


#### Extract timestamp

In [4]:
exportFolder = "../00 dataset/"

In [5]:
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
from datetime import datetime

def getTarinSet(df):
    start = datetime(2014, 1, 1)
    end   = datetime(2015, 7, 31)
    df = df.loc[(df.Date >= start) & (df.Date <= end)]
    return df

def getValidSet(df):
    start = datetime(2015, 8, 1)
    end   = datetime(2015, 9, 30)
    df = df.loc[(df.Date >= start) & (df.Date <= end)]
    return df

def getTestSet(df):
    start = datetime(2015, 10, 1)
    end   = datetime(2016, 1, 1)
    df = df.loc[(df.Date >= start) & (df.Date <= end)]
    return df

In [8]:
trainDate = getTarinSet(df)[['Date']]
validDate = getValidSet(df)[['Date']]
testDate  = getTestSet(df)[['Date']]

In [17]:
print(len(trainDate), len(validDate), len(testDate))

398 42 64


In [22]:
# save as .npy
with open(exportFolder+'train_timestamp.npy', 'wb') as f:
    np.save(f, trainDate['Date'].to_numpy())
with open(exportFolder+'valid_timestamp.npy', 'wb') as f:
    np.save(f, validDate['Date'].to_numpy())
with open(exportFolder+'test_timestamp.npy', 'wb') as f:
    np.save(f, testDate['Date'].to_numpy())

In [18]:
# save as csv (with index)
trainDate.to_csv(exportFolder+'train_timestamp.csv',  header=True)
validDate.to_csv(exportFolder+'valid_timestamp.csv',  header=True)
testDate.to_csv(exportFolder+'test_timestamp.csv',  header=True)

#### Get all files of a directory

In [59]:
from os import walk

_, _, filenames = next(walk(abs_path))

In [60]:
print(len(filenames))
filenames[:5]

88


['AAPL.txt', 'ABB.txt', 'ABBV.txt', 'AEP.txt', 'AGFS.txt']

#### Extract all label

In [61]:
exportFolder = "../00 dataset/"

In [62]:
# this function is called only when to fill missing value, complexity O(n^2)
def fill(missing_df, dataset=trainDate, value_to_fill=[0]):
    for i, row in dataset.iterrows():
        # if find missing a date, insert in
        hasDate = False
        for d in missing_df['Date']:
            if d == row['Date']:
                hasDate = True
        if hasDate is False:
            missing_df.loc[-1] = [row['Date']] + value_to_fill # insert all 0
            missing_df.index = missing_df.index + 1  # shifting index
            missing_df = missing_df.sort_index()

    # make its order by date
    return missing_df.sort_values('Date')

In [ ]:
# test code
# read missing_df
# missing_df = pd.read_csv(abs_path + "\\" + 'AGFS.txt', delimiter = "\t", names=["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"])
# missing_df = missing_df.iloc[::-1]
# missing_df = missing_df.reset_index(drop=True)
# missing_df = missing_df[["Date","Close"]]

# # divide into train, valid, test
# missing_df['Date'] = pd.to_datetime(missing_df['Date'])
# missing_df = getTarinSet(missing_df)

# res = fill(missing_df)
# res

幾乎所有trainset的len = 398，有173,217,0三支不合群  
幾乎所有validset的len = 42，有1支為0  
幾乎所有testset的len = 64，有1支為0
``` 
AGFS.txt
BABA.txt
GMRE.txt
```

In [92]:
# label_array = np.array([])
train_labels, valid_labels, test_labels = [], [], []

for filename in filenames:
    # read a single file
    df = pd.read_csv(abs_path + "\\" + filename, delimiter = "\t", names=["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"])
    df = df.iloc[::-1]
    df = df.reset_index(drop=True)
    df = df[["Date","Close"]]
    
    # divide into train, valid, test
    df['Date'] = pd.to_datetime(df['Date'])
    df_train = getTarinSet(df)
    df_valid = getValidSet(df)
    df_test  = getTestSet(df)
    
    # insert the missing rows with 0 (for 3 stocks)
    if len(df_train) in [173,217,0]:
        df_train = fill(df_train, trainDate) # default param value_to_fill is [0]
    if len(df_valid) == 0:
        df_valid = fill(df_valid, validDate)
    if len(df_test) == 0:
        df_test = fill(df_test, testDate)
    
    # init column with derived value np.nan
    df_train['Label'] = np.nan
    df_valid['Label'] = np.nan
    df_test['Label']  = np.nan
    
    df_train.reset_index(drop=True, inplace=True)
    df_valid.reset_index(drop=True, inplace=True)
    df_test.reset_index(drop=True, inplace=True)
    
    # apply label fram index 1 ~ last, if price ↑ then assign 1
    for i in range(1, len(df_train)):
        df_train.loc[i, 'Label'] = 1 if df_train.loc[i, 'Close'] >= df_train.loc[i-1, 'Close'] else 0
    for i in range(1, len(df_valid)):
        df_valid.loc[i, 'Label'] = 1 if df_valid.loc[i, 'Close'] >= df_valid.loc[i-1, 'Close'] else 0
    for i in range(1, len(df_test)):
        df_test.loc[i, 'Label']  = 1 if df_test.loc[i, 'Close']  >= df_test.loc[i-1, 'Close'] else 0
        
    train_labels.append(df_train['Label'].to_numpy())
    valid_labels.append(df_valid['Label'].to_numpy())
    test_labels.append(df_test['Label'].to_numpy())

train_labels = np.array(train_labels)
valid_labels = np.array(valid_labels)
test_labels  = np.array(test_labels)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [93]:
print(train_labels.shape)
print(valid_labels.shape)
print(test_labels.shape)

(88, 398)
(88, 42)
(88, 64)


In [94]:
# save as .npy
with open(exportFolder+'train_label.npy', 'wb') as f:
    np.save(f, train_labels)
with open(exportFolder+'valid_label.npy', 'wb') as f:
    np.save(f, valid_labels)
with open(exportFolder+'test_label.npy', 'wb') as f:
    np.save(f, test_labels)

### 以下是測試單一支股票的code
現在用不到了 不要執行到唷~

#### Extract timestamp

In [45]:
timestamp = pd.to_datetime(df['Date'])
timestamp.to_csv(exportFolder+'timestamp.csv',  header=True)

In [46]:
with open(exportFolder+'timestamp.npy', 'wb') as f:
    np.save(f, timestamp.to_numpy())

In [47]:
with open(exportFolder+'timestamp.npy', 'rb') as f:
    a = np.load(f)

In [48]:
a

array(['2012-09-05T00:00:00.000000000', '2012-09-06T00:00:00.000000000',
       '2012-09-07T00:00:00.000000000', ...,
       '2017-08-30T00:00:00.000000000', '2017-08-31T00:00:00.000000000',
       '2017-09-01T00:00:00.000000000'], dtype='datetime64[ns]')

#### Extract label

In [50]:
import numpy as np

# init column with derived value np.nan
df['Label'] = np.nan
df.head(2)

,Date,High,Low,Close,Label
0,2012-09-05,0.107768,0.109047,0.097979,NaN
1,2012-09-06,0.111639,0.120094,0.107725,NaN


In [51]:
# apply label fram index 5 ~ last
# if price ↑ then assign 1
for i in range(1, len(df)):
    df.loc[i, 'Label'] = 1 if df.loc[i, 'Close'] >= df.loc[i-1, 'Close'] else 0

In [52]:
df.head(10)

,Date,High,Low,Close,Label
0,2012-09-05,0.107768,0.109047,0.097979,NaN
1,2012-09-06,0.111639,0.120094,0.107725,1.0
2,2012-09-07,0.109697,0.116947,0.105966,0.0
3,2012-09-10,0.106800,0.111420,0.076953,0.0
4,2012-09-11,0.110742,0.119075,0.096363,1.0
5,2012-09-12,0.117272,0.122382,0.099094,1.0
6,2012-09-13,0.119309,0.132744,0.115013,1.0
7,2012-09-14,0.118095,0.129471,0.114741,0.0
8,2012-09-17,0.119705,0.120425,0.112116,0.0
9,2012-09-18,0.106942,0.110817,0.101470,0.0


In [53]:
df.tail(10)

,Date,High,Low,Close,Label
1247,2017-08-21,0.000000,0.002476,-0.015175,0.0
1248,2017-08-22,0.006488,0.017747,0.005152,1.0
1249,2017-08-23,-0.004444,0.004318,-0.005633,0.0
1250,2017-08-24,0.002813,0.004751,-0.008939,0.0
1251,2017-08-25,0.002386,0.008099,0.000000,1.0
1252,2017-08-28,0.001752,0.013387,0.000438,1.0
1253,2017-08-29,-0.008485,0.010219,-0.009104,0.0
1254,2017-08-30,0.005463,0.006016,-0.001842,1.0
1255,2017-08-31,0.001775,0.007163,0.000796,1.0
1256,2017-09-01,0.004878,0.005732,-0.002256,0.0


In [54]:
label = df['Label']
label.to_csv(exportFolder+'label.csv',  header=True)

In [55]:
with open(exportFolder+'label.npy', 'wb') as f:
    np.save(f, label.to_numpy())

In [56]:
with open(exportFolder+'label.npy', 'rb') as f:
    a = np.load(f)
a

array([nan,  1.,  0., ...,  1.,  1.,  0.])

#### Extract price data

In [58]:
price = df[["High","Low","Close"]]
price.to_csv(exportFolder+'price.csv',  header=True)

In [59]:
with open(exportFolder+'price.npy', 'wb') as f:
    np.save(f, price.to_numpy())

In [60]:
with open(exportFolder+'price.npy', 'rb') as f:
    a = np.load(f)
a

array([[ 0.107768,  0.109047,  0.097979],
       [ 0.111639,  0.120094,  0.107725],
       [ 0.109697,  0.116947,  0.105966],
       ...,
       [ 0.005463,  0.006016, -0.001842],
       [ 0.001775,  0.007163,  0.000796],
       [ 0.004878,  0.005732, -0.002256]])